# DSLab Homework 1 - Data Science with CO2

## Hand-in Instructions

- __Due: 23.03.2021 23h59 CET__
- `git push` your final verion to the master branch of your group's Renku repository before the due
- check if `Dockerfile`, `environment.yml` and `requirements.txt` are properly written
- add necessary comments and discussion to make your codes readable

## Carbosense

The project Carbosense establishes a uniquely dense CO2 sensor network across Switzerland to provide near-real time information on man-made emissions and CO2 uptake by the biosphere. The main goal of the project is to improve the understanding of the small-scale CO2 fluxes in Switzerland and concurrently to contribute to a better top-down quantification of the Swiss CO2 emissions. The Carbosense network has a spatial focus on the City of Zurich where more than 50 sensors are deployed. Network operations started in July 2017.

<img src="http://carbosense.wdfiles.com/local--files/main:project/CarboSense_MAP_20191113_LowRes.jpg" width="500">

<img src="http://carbosense.wdfiles.com/local--files/main:sensors/LP8_ZLMT_3.JPG" width="156">  <img src="http://carbosense.wdfiles.com/local--files/main:sensors/LP8_sensor_SMALL.jpg" width="300">

## Description of the homework

In this homework, we will curate a set of **CO2 measurements**, measured from cheap but inaccurate sensors, that have been deployed in the city of Zurich from the Carbosense project. The goal of the exercise is twofold: 

1. Learn how to deal with real world sensor timeseries data, and organize them efficiently using python dataframes.

2. Apply data science tools to model the measurements, and use the learned model to process them (e.g., detect drifts in the sensor measurements). 

The sensor network consists of 46 sites, located in different parts of the city. Each site contains three different sensors measuring (a) **CO2 concentration**, (b) **temperature**, and (c) **humidity**. Beside these measurements, we have the following additional information that can be used to process the measurements: 

1. The **altitude** at which the CO2 sensor is located, and the GPS coordinates (latitude, longitude).

2. A clustering of the city of Zurich in 17 different city **zones** and the zone in which the sensor belongs to. Some characteristic zones are industrial area, residential area, forest, glacier, lake, etc.

## Prior knowledge

The average value of the CO2 in a city is approximately 400 ppm. However, the exact measurement in each site depends on parameters such as the temperature, the humidity, the altitude, and the level of traffic around the site. For example, sensors positioned in high altitude (mountains, forests), are expected to have a much lower and uniform level of CO2 than sensors that are positioned in a business area with much higher traffic activity. Moreover, we know that there is a strong dependence of the CO2 measurements, on temperature and humidity.

Given this knowledge, you are asked to define an algorithm that curates the data, by detecting and removing potential drifts. **The algorithm should be based on the fact that sensors in similar conditions are expected to have similar measurements.** 

## To start with

The following csv files in the `../data/carbosense-raw/` folder will be needed: 

1. `CO2_sensor_measurements.csv`
    
   __Description__: It containts the CO2 measurements `CO2`, the name of the site `LocationName`, a unique sensor identifier `SensorUnit_ID`, and the time instance in which the measurement was taken `timestamp`.
    
2. `temperature_humidity.csv`

   __Description__: It contains the temperature and the humidity measurements for each sensor identifier, at each timestamp `Timestamp`. For each `SensorUnit_ID`, the temperature and the humidity can be found in the corresponding columns of the dataframe `{SensorUnit_ID}.temperature`, `{SensorUnit_ID}.humidity`.
    
3. `sensor_metadata.csv`

   __Description__: It contains the name of the site `LocationName`, the zone index `zone`, the altitude in meters `altitude`, the longitude `lon`, and the latitude `lat`. 

Import the following python packages:

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import plotly.express as px
import plotly.graph_objects as go
import os

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt

from causalimpact import CausalImpact

from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

!git lfs pull

pd.options.mode.chained_assignment = None

## PART I: Handling time series with pandas (10 points)

### a) **8/10**

Merge the `CO2_sensor_measurements.csv`, `temperature_humidity.csv`, and `sensors_metadata.csv`, into a single dataframe. 

* The merged dataframe contains:
    - index: the time instance `timestamp` of the measurements
    - columns: the location of the site `LocationName`, the sensor ID `SensorUnit_ID`, the CO2 measurement `CO2`, the `temperature`, the `humidity`, the `zone`, the `altitude`, the longitude `lon` and the latitude `lat`.

| timestamp | LocationName | SensorUnit_ID | CO2 | temperature | humidity | zone | altitude | lon | lat |
|:---------:|:------------:|:-------------:|:---:|:-----------:|:--------:|:----:|:--------:|:---:|:---:|
|    ...    |      ...     |      ...      | ... |     ...     |    ...   |  ... |    ...   | ... | ... |



* For each measurement (CO2, humidity, temperature), __take the average over an interval of 30 min__. 

* If there are missing measurements, __interpolate them linearly__ from measurements that are close by in time.

__Hints__: The following methods could be useful

1. ```python 
pandas.DataFrame.resample()
``` 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html
    
2. ```python
pandas.DataFrame.interpolate()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
    
3. ```python
pandas.DataFrame.mean()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html
    
4. ```python
pandas.DataFrame.append()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html

In [ ]:
#load data.
CO2_sensor_measurements = pd.read_csv('../data/carbosense-raw/CO2_sensor_measurements.csv', sep='\t')
temperature_humidity = pd.read_csv('../data/carbosense-raw/temperature_humidity.csv', sep='\t')
sensors_metadata = pd.read_csv('../data/carbosense-raw/sensors_metadata.csv', sep='\t')

In [ ]:
# Check the missing data.
print('Missing values in CO2_sensor_measurements:', CO2_sensor_measurements.isnull().sum().sum())
print('Missing values in temperature_humidity:', temperature_humidity.isnull().sum().sum())
print('Missing values in sensors_metadata:', sensors_metadata.isnull().sum().sum())

In [ ]:
# fill the missing data.
temperature_humidity.interpolate(method='linear', limit_direction='both', inplace=True)

In [ ]:
# Check the missing data again.
print('Missing values in temperature_humidity:', temperature_humidity.isnull().sum().sum())

In [ ]:
# Melt the dataframe so the columns are converted to rows including timestamp, temp/hum and their respective value.
melted_df = temperature_humidity.melt(id_vars=["Timestamp"], var_name="Temp/Hum", value_name="Value")

# Add a new column with the IDs and remove IDs from temp/hum.
new = melted_df['Temp/Hum'].str[:4].rename('SensorUnit_ID')
new_melted_df = pd.concat([melted_df, new], axis=1)
new_melted_df['Temp/Hum'] = new_melted_df['Temp/Hum'].str[5:]

# Pivot the table to get the correct format.
new_temperature_humidity = new_melted_df.pivot_table(index=['Timestamp', 'SensorUnit_ID'], columns='Temp/Hum', values='Value')\
    .rename(columns={'humidity':'Humidity', 'temperature':'Temperature'})\
    .sort_values(by=['SensorUnit_ID', 'Timestamp']).reset_index()

In [ ]:
# Convert timestamp columns to correct date/time format and set as indexes.
CO2_sensor_measurements['Timestamp'] = pd.to_datetime(CO2_sensor_measurements['timestamp'])
new_temperature_humidity['Timestamp'] = pd.to_datetime(new_temperature_humidity['Timestamp'])
CO2_sensor_measurements.set_index('Timestamp', inplace=True)
new_temperature_humidity.set_index('Timestamp', inplace=True)

In [ ]:
# Resample CO2 df into 30 minute intervals, keep IDs & Location Names and average over CO2-values.
CO2_resampled = CO2_sensor_measurements.groupby(['SensorUnit_ID', 'LocationName'])\
    .resample('30T', fill_method='ffill').mean()
CO2_resampled= CO2_resampled.drop(columns='SensorUnit_ID').reset_index()
CO2_resampled.CO2 = CO2_resampled.CO2.interpolate(method='linear')

In [ ]:
# Resample Temp/Hum df into 30 minute intervals, keep IDs and average over Hum/Temp values.
t_h_resampled = new_temperature_humidity.groupby(['SensorUnit_ID'])\
    .resample('30T', fill_method='ffill').mean().reset_index()
t_h_resampled.Humidity = t_h_resampled.Humidity.interpolate(method='linear')
t_h_resampled.Temperature = t_h_resampled.Temperature.interpolate(method='linear')

In [ ]:
# Converting the SensorUnit_ID columns to int to make sure that merging works.
CO2_resampled['SensorUnit_ID'] = CO2_resampled['SensorUnit_ID'].astype(int)
t_h_resampled['SensorUnit_ID'] = t_h_resampled['SensorUnit_ID'].astype(int)

In [ ]:
# Merging data.
measures = pd.merge(t_h_resampled, CO2_resampled, how='right', 
                    on=['Timestamp', 'SensorUnit_ID']).reset_index()
measures = pd.merge(measures.set_index('LocationName'), 
                    sensors_metadata.set_index('LocationName'), 
                    left_index=True, right_index=True)\
    .drop(columns=['index']).reset_index().set_index('Timestamp')
measures = measures[['LocationName','SensorUnit_ID','CO2','Temperature','Humidity','zone','altitude','lon','lat']]

In [ ]:
measures.head()

### b) **2/10** 

Export the curated and ready to use timeseries to a csv file, and properly push the merged csv to Git LFS.

In [ ]:
measures.to_csv(r'../data/carbosense-raw/export_measures.csv', index = True, header=True)

## PART II: Data visualization (15 points)

### a) **5/15** 
Group the sites based on their altitude, by performing K-means clustering. 
- Find the optimal number of clusters using the [Elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)). 
- Wite out the formula of metric you use for Elbow curve. 
- Perform clustering with the optimal number of clusters and add an additional column `altitude_cluster` to the dataframe of the previous question indicating the altitude cluster index. 
- Report your findings.

__Note__: [Yellowbrick](http://www.scikit-yb.org/) is a very nice Machine Learning Visualization extension to scikit-learn, which might be useful to you. 

formula of metric: mean sum of squared distances to centers.

In [ ]:
# cluster using Yellowbrick.
altitudes = np.asarray(measures['altitude'].tolist())
altitudes = altitudes.reshape(-1, 1)
visualizer = KElbowVisualizer(KMeans())
visualizer.fit(altitudes) 
visualizer.show()  
elbow_value = visualizer.elbow_value_

best k is 4 !

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(altitudes)

measures['altitude_cluster'] = kmeans.predict(altitudes)

finding: Four clusters are the best.

### b) **4/15** 

Use `plotly` (or other similar graphing libraries) to create an interactive plot of the monthly median CO2 measurement for each site with respect to the altitude. 

Add proper title and necessary hover information to each point, and give the same color to stations that belong to the same altitude cluster.

In [ ]:
CO2_median = measures.groupby(['LocationName', 'altitude', 'altitude_cluster']).agg({'CO2':'median'}).reset_index()   
fig = px.scatter(CO2_median, x='altitude', y='CO2', color='altitude_cluster')

fig.update_layout(title=go.layout.Title(text="Monthly median CO2 measurement for each site w.r.t the altitude"))

fig.show()

### c) **6/15**

Use `plotly` (or other similar graphing libraries) to plot an interactive time-varying density heatmap of the mean daily CO2 concentration for all the stations. Add proper title and necessary hover information.

__Hints:__ Check following pages for more instructions:
- [Animations](https://plotly.com/python/animations/)
- [Density Heatmaps](https://plotly.com/python/mapbox-density-heatmaps/)

In [ ]:
day = measures.index.day

measures['day'] = day

CO2_median_daily = measures.groupby(['day','LocationName', 'altitude', 'altitude_cluster','lat','lon'])\
    .agg({'CO2':'median'}).reset_index()

center = dict(lat = CO2_median_daily['lat'].mean(), lon = CO2_median_daily['lon'].mean())
                                                           
fig2 = px.density_mapbox(CO2_median_daily, lat='lat', lon='lon',z='CO2',animation_frame="day",hover_data=['LocationName'],radius=20,
                        center=center, zoom=10,
                        mapbox_style="stamen-terrain")

fig2.update_layout(title=go.layout.Title(text="Mean daily CO2 concentration for all the stations"))

fig2.show()

## PART III: Model fitting for data curation (35 points)

### a) **2/35**

The domain experts in charge of these sensors report that one of the CO2 sensors `ZSBN` is exhibiting a drift on Oct. 24. Verify the drift by visualizing the CO2 concentration of the drifting sensor and compare it with some other sensors from the network. 

In [ ]:
# Define Dataframes for different locations for the Oct.

data_drift = measures[measures['LocationName'] == 'ZSBN'] 
data_drift_sample = data_drift.loc['2017-10-1':'2017-10-31',].reset_index()

data_drift_1 = measures[measures['LocationName'] == 'ZHBG']
data_drift_sample_1 = data_drift_1.loc['2017-10-1':'2017-10-31'].reset_index()

data_drift_2 = measures[measures['LocationName'] == 'ZWCH']
data_drift_sample_2 = data_drift_2.loc['2017-10-1':'2017-10-31'].reset_index()


# Plot results
figure = plt.figure()
plt.plot(data_drift_sample.Timestamp, data_drift_sample.CO2)
plt.plot(data_drift_sample_1.Timestamp, data_drift_sample_1.CO2)
plt.plot(data_drift_sample_2.Timestamp, data_drift_sample_2.CO2)
plt.xlabel('Time')
plt.ylabel('CO2')
plt.legend(('ZSBN', 'ZHBG', 'ZWCH'), title='LocationName')

It can be seen from the figure that Oct 24 may have drifted.

### b) **8/35**

The domain experts ask you if you could reconstruct the CO2 concentration of the drifting sensor had the drift not happened. You decide to:
- Fit a linear regression model to the CO2 measurements of the site, by considering as features the covariates not affected by the malfunction (such as temperature and humidity)
- Create an interactive plot with `plotly` (or other similar graphing libraries):
    - the actual CO2 measurements
    - the values obtained by the prediction of the linear model for the entire month of October
    - the __confidence interval__ obtained from cross validation
- What do you observe? Report your findings.

__Note:__ Cross validation on time series is different from that on other kinds of datasets. The following diagram illustrates the series of training sets (in orange) and validation sets (in blue). For more on time series cross validation, there are a lot of interesting articles available online. scikit-learn provides a nice method [`sklearn.model_selection.TimeSeriesSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).

![ts_cv](https://player.slideplayer.com/86/14062041/slides/slide_28.jpg)

In [ ]:
# DataFrame for ZSBN location at all time 
measures_drift =  measures[measures['LocationName'] == 'ZSBN']

# DataFrame for ZSBN location at Oct 1 - Oct 23 (before drift) 
measures_drift_by24 = measures_drift.loc['2017-10-01':'2017-10-23'].reset_index()

# Data for linear regression
X = pd.DataFrame(measures_drift_by24, columns=['Humidity', 'Temperature'])
y = measures_drift_by24['CO2']
X_all = pd.DataFrame(measures_drift, columns=['Humidity', 'Temperature'])

# Linear regression and predict 
reg = LinearRegression().fit(X, y)
CO2_predict = reg.predict(X_all)
measures_drift['CO2_predict'] = CO2_predict
measures_drift.reset_index()

In [ ]:
# Calculate score and confidential interval by cross validation
tscv = TimeSeriesSplit(n_splits=5)   
reg = LinearRegression()
scores = []

scores = cross_validate(reg, X, y, cv=tscv, scoring= 'neg_root_mean_squared_error')['test_score']
scores = np.abs(scores)
sigma = np.mean(scores)

# Confidential interval for 0.95
measures_drift['confidence_low'] = measures_drift['CO2_predict'] - 1.96*sigma
measures_drift['confidence_up'] = measures_drift['CO2_predict'] + 1.96*sigma

In [ ]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2'],mode='lines+markers',name='Original Data'))
fig3.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2_predict'],mode='lines+markers',name='Predict Value'))
fig3.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_low'],mode='lines',name='Confidence Low'))
fig3.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_up'],mode='lines',name='Confidence Up'))
fig3.update_layout(title=go.layout.Title(text="Predict CO2 value and confidential interval"))
fig3.show()

We found that the observation on Oct 24 have a decrease, and thus we conclude that the sensor have a drift on Oct 24

### c) **10/35**

In your next attempt to solve the problem, you decide to exploit the fact that the CO2 concentrations, as measured by the sensors __experiencing similar conditions__, are expected to be similar.

- Find the sensors sharing similar conditions with `ZSBN`. Explain your definition of "similar condition".
- Fit a linear regression model to the CO2 measurements of the site, by considering as features:
    - the information of provided by similar sensors
    - the covariates associated with the faulty sensors that were not affected by the malfunction (such as temperature and humidity).
- Create an interactive plot with `plotly` (or other similar graphing libraries):
    - the actual CO2 measurements
    - the values obtained by the prediction of the linear model for the entire month of October
    - the __confidence interval__ obtained from cross validation
- What do you observe? Report your findings.

In [ ]:
# choose the similar sensors
# choose condition: the same cluster, lon gap < lon's std, lat gap < lat's std,
drift_cluster = measures[measures['LocationName'] == 'ZSBN']['altitude_cluster'][0]
drift_lat = measures[measures['LocationName'] == 'ZSBN']['lat'][0]
drift_lon = measures[measures['LocationName'] == 'ZSBN']['lon'][0]
std_lat = np.std(measures.lat)
std_lon = np.std(measures.lon)
similar_drift = measures[measures['altitude_cluster'] == drift_cluster].reset_index().set_index('Timestamp')
similar_drift = similar_drift[similar_drift['lat'] <= drift_lat + std_lat].reset_index().set_index('Timestamp')
similar_drift = similar_drift[similar_drift['lat'] >= drift_lat - std_lat].reset_index().set_index('Timestamp')
similar_drift = similar_drift[similar_drift['lon'] <= drift_lon + std_lon].reset_index().set_index('Timestamp')
similar_drift = similar_drift[similar_drift['lon'] >= drift_lon - std_lon].reset_index().set_index('Timestamp')
similar_drift_by24 = similar_drift[similar_drift['LocationName'] != 'ZSBN'].reset_index().set_index('Timestamp')
similar_drift_by24 = pd.concat( [measures_drift_by24.set_index('Timestamp'), similar_drift_by24], axis=0 )

In [ ]:
# Linear regression and predict
X = pd.DataFrame(similar_drift_by24, columns=['Humidity', 'Temperature','altitude'])
y = similar_drift_by24['CO2']

X_all = pd.DataFrame(measures_drift, columns=['Humidity', 'Temperature','altitude'])

reg = LinearRegression().fit(X, y)
CO2_predict = reg.predict(X_all)

measures_drift['CO2_predict'] = CO2_predict
measures_drift.reset_index()

In [ ]:
# Calculate score and confidential interval by cross validation
tscv = TimeSeriesSplit(n_splits=5)   
scores = []
scores = cross_validate(LinearRegression(), X, y, cv=tscv, scoring= 'neg_root_mean_squared_error')['test_score']
scores = np.abs(scores)
sigma = np.mean(scores)

# Confidential interval for 0.95
measures_drift['confidence_low'] = measures_drift['CO2_predict'] - 1.96*sigma
measures_drift['confidence_up'] = measures_drift['CO2_predict'] + 1.96*sigma

In [ ]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2'],mode='lines+markers',name='Original Data'))
fig4.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2_predict'],mode='lines+markers',name='Predict Value'))
fig4.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_low'],mode='lines',name='Confidence Low'))
fig4.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_up'],mode='lines',name='Confidence Up'))
fig4.update_layout(title=go.layout.Title(text="CO2 value and predict CO2 value from linear regression"))
fig4.show()

After the data augment, the confidencial interval become larger. We stiil find after Oct 24, the data is beyond  confidencial interval , so we conclude that the sensor have a drift at Oct 24.

### d) **10/35**

Now, instead of feeding the model with all features, you want to do something smarter by using linear regression with fewer features.

- Start with the same sensors and features as in question c)
- Leverage at least two different feature selection methods
- Create similar interactive plot as in question c)
- Describe the methods you choose and report your findings

We choose two methods: Feature selection and PCA

In [ ]:
# method 1: Feature selection based on coefficient
# pick the top k features with the highest coefficient
feature_list = ['Humidity', 'Temperature','altitude']
num_feature = 2

X = pd.DataFrame(similar_drift_by24, columns=feature_list)
y = similar_drift_by24['CO2']

X_all = pd.DataFrame(measures_drift, columns=feature_list)

reg = LinearRegression().fit(X, y)
coef = np.abs(reg.coef_)

idx =  np.argsort(coef)[-num_feature:]

new_feature_list = [feature_list[i] for i in idx]

In [ ]:
# Linear regression and predict
X = pd.DataFrame(similar_drift_by24, columns=new_feature_list)
y = similar_drift_by24['CO2']

X_all = pd.DataFrame(measures_drift, columns=new_feature_list)

reg = LinearRegression().fit(X, y)
CO2_predict = reg.predict(X_all)

measures_drift['CO2_predict'] = CO2_predict
measures_drift.reset_index()

In [ ]:
# Calculate score and confidential interval by cross validation
tscv = TimeSeriesSplit(n_splits=5)   

scores = []

scores = cross_validate(LinearRegression(), X, y, cv=tscv, scoring= 'neg_root_mean_squared_error')['test_score']

scores = np.abs(scores)
sigma = np.mean(scores)

# Confidential interval for 0.95
measures_drift['confidence_low'] = measures_drift['CO2_predict'] - 1.96*sigma
measures_drift['confidence_up'] = measures_drift['CO2_predict'] + 1.96*sigma

In [ ]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2'],mode='lines+markers',name='Original Data'))
fig5.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2_predict'],mode='lines+markers',name='Predict Value'))
fig5.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_low'],mode='lines',name='Confidence Low'))
fig5.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_up'],mode='lines',name='Confidence Up'))
fig5.show()

In [ ]:
# method 2: Feature selection based on PCA
from sklearn.decomposition import PCA

feature_list = ['Humidity', 'Temperature','altitude']
pca = PCA(n_components=2)


X = pd.DataFrame(similar_drift_by24, columns=feature_list)
x = pca.fit_transform(X)
y = similar_drift_by24['CO2']

reg = LinearRegression().fit(x, y)

X_all = pd.DataFrame(measures_drift, columns=feature_list)
x_all = pca.transform(X_all)


coef = np.abs(reg.coef_)

idx =  np.argsort(coef)[-num_feature:]

new_feature_list = [feature_list[i] for i in idx]

In [ ]:
# Linear regression and predict
CO2_predict = reg.predict(x_all)

measures_drift['CO2_predict'] = CO2_predict
measures_drift.reset_index()

In [ ]:
# Calculate score and confidential interval by cross validation
tscv = TimeSeriesSplit(n_splits=5)   

scores = []

scores = cross_validate(LinearRegression(), x, y, cv=tscv, scoring= 'neg_root_mean_squared_error')['test_score']

scores = np.abs(scores)
sigma = np.mean(scores)

# Confidential interval for 0.95
measures_drift['confidence_low'] = measures_drift['CO2_predict'] - 1.96*sigma
measures_drift['confidence_up'] = measures_drift['CO2_predict'] + 1.96*sigma

In [ ]:
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2'],mode='lines+markers',name='Original Data'))
fig6.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['CO2_predict'],mode='lines+markers',name='Predict Value'))
fig6.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_low'],mode='lines',name='Confidence Low'))
fig6.add_trace(go.Scatter(x=measures_drift.index, y=measures_drift['confidence_up'],mode='lines',name='Confidence Up'))
fig6.show()

We found that the confidence interval did not change significantly after using Feature selection and PCA.

### e) **5/35**

Eventually, you'd like to try something new - __Bayesian Structural Time Series Modelling__ - to reconstruct counterfactual values, that is, what the CO2 measurements of the faulty sensor should have been, had the malfunction not happened on October 24. You will use:
- the information of provided by similar sensors - the ones you identified in question c)
- the covariates associated with the faulty sensors that were not affected by the malfunction (such as temperature and humidity).

To answer this question, you can choose between a Python port of the CausalImpact package (such as https://github.com/dafiti/causalimpact) or the original R version (https://google.github.io/CausalImpact/CausalImpact.html) that you can run in your notebook via an R kernel (https://github.com/IRkernel/IRkernel).

Before you start, watch first the [presentation](https://www.youtube.com/watch?v=GTgZfCltMm8) given by Kay Brodersen (one of the creators of the causal impact implementation in R), and this introductory [ipython notebook](http://nbviewer.jupyter.org/github/dafiti/causalimpact/blob/master/examples/getting_started.ipynb) with examples of how to use the python package.

- Report your findings:
    - Is the counterfactual reconstruction of CO2 measurements significantly different from the observed measurements?
    - Can you try to explain the results?

In [ ]:
# Use CausalImpact to analyze data
pre_post = measures_drift.loc['2017-10-1':'2017-10-23',].index.size
total = measures_drift.index.size
pre_period = [0, pre_post - 1]
post_period = [pre_post, total - 1]

data = pd.DataFrame({'x0': measures_drift['Temperature'], 'x1': measures_drift['Humidity'], 'y': measures_drift['CO2']}, columns=['y', 'x0', 'x1']).reset_index().drop(['Timestamp'], axis=1)
ci = CausalImpact(data, pre_period, post_period)
ci.plot()

As you can see from the graph, The drift starts at Oct 24.

# That's all, folks!